In [1]:
import pandas as pd

In [2]:
train_df=pd.read_csv("train.csv")

In [3]:
print(train_df)

                     id        anchor                  target context  score
0      37d61fd2272659b1     abatement  abatement of pollution     A47   0.50
1      7b9652b17b68b7a4     abatement          act of abating     A47   0.75
2      36d72442aefd8232     abatement         active catalyst     A47   0.25
3      5296b0c19e1ce60e     abatement     eliminating process     A47   0.50
4      54c1e3b9184cb5b6     abatement           forest region     A47   0.00
...                 ...           ...                     ...     ...    ...
36468  8e1386cbefd7f245  wood article          wooden article     B44   1.00
36469  42d9e032d1cd3242  wood article              wooden box     B44   0.50
36470  208654ccb9e14fa3  wood article           wooden handle     B44   0.50
36471  756ec035e694722b  wood article         wooden material     B44   0.75
36472  8d135da0b55b8c88  wood article        wooden substrate     B44   0.50

[36473 rows x 5 columns]


In [29]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Sample embeddings (replace with your actual embeddings)
embeddings = {
    "cat": [0.5, 0.8, 0.3, ...],  # Embedding in higher dimensions
    "dog": [0.7, 0.9, 0.2, ...],  # Embedding in higher dimensions
    "fish": [0.3, 0.6, 0.1, ...]  # Embedding in higher dimensions
}

# Convert embeddings to a list
embedding_list = list(embeddings.values())

# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embedding_list)

# Plot reduced embeddings
plt.figure(figsize=(6, 4))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], color='blue')

# Annotate points with words
for word, embedding in zip(embeddings.keys(), reduced_embeddings):
    plt.annotate(word, embedding, textcoords="offset points", xytext=(0,10), ha='center')

# Set labels and title
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Visual Representation of Embeddings (PCA)')

# Show plot
plt.grid(True)
plt.show()


TypeError: float() argument must be a string or a real number, not 'ellipsis'

In [10]:
import nltk

In [11]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import  string

In [12]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stemmer=PorterStemmer()
def preprocess_step(text):
    tokens=word_tokenize(text)
    tokens=[token.lower() for token in tokens]
    tokens=[token for token in tokens  if token not in string.punctuation]
    stop_words=set(stopwords.words('english'))
    tokens=[token for token in tokens  if token not in stop_words]
    token=[stemmer.stem(token) for token in tokens]
    preprocess_step=' '.join(tokens)
    return preprocess_step


train_df['anchor_cleaned']=train_df['anchor'].apply(preprocess_step)
train_df['target_cleaned']=train_df['target'].apply(preprocess_step)
print(train_df[['anchor',   'anchor_cleaned', 'target', 'target_cleaned']].head())


      anchor anchor_cleaned                  target       target_cleaned
0  abatement      abatement  abatement of pollution  abatement pollution
1  abatement      abatement          act of abating          act abating
2  abatement      abatement         active catalyst      active catalyst
3  abatement      abatement     eliminating process  eliminating process
4  abatement      abatement           forest region        forest region


After this process we need to convert preprocess data intoo numerical feature so that it can be used as input for machine learning model  converting in numerical there are several techniques for feature extraction

Bag of Words (BoW): Represent each document as a vector where each element represents the frequency of a word in the document. This approach ignores the order of words.

TF-IDF (Term Frequency-Inverse Document Frequency): Similar to BoW, but it also takes into account the importance of words in the corpus by weighting terms that appear frequently across documents lower.

Word Embeddings: Represent words as dense vectors in a continuous vector space. Word embeddings capture semantic meaning and relationships between words.

Doc2Vec: An extension of word embeddings, Doc2Vec also learns embeddings for entire documents.

BERT Embeddings: Pretrained transformer-based language models like BERT can generate contextualized word embeddings that capture the meaning of words in the context of the entire sentence.

let's start by loading a pretrained BERT model and tokenizing the text data. We'll use the Hugging Face's Transformers library for this purpose.

In [20]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     ------------------ -------------------- 61.4/130.7 kB 1.7 MB/s eta 0:00:01
     ----------------------------------- -- 122.9/130.7 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 130.7/130.7 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.5 MB 5.5 MB/s eta 0:00:02
   - -------------------------------------- 0.2/8.5 MB 2.8 MB/s eta 0:00:04
   - -------------------------------------- 0.4/8.5 MB 3.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/8.5 MB 3.3 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/8.5 MB 3.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.5 MB 3.7 MB/s eta 0:00:03
   ----- ---------------------------------- 1.2/8.5 MB 3.8 MB/s eta 0:00:02
   -----

In [25]:
from transformers import BertTokenizer, BertModel
import torch

In [27]:
import torch
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def tokenize_and_pad(text):
    tokens = tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    return tokens
def encode_with_bert(tokens):
    with torch.no_grad():
        outputs = model(**tokens)
        pooled_output = outputs.pooler_output
    return pooled_output

def extract_features(data):
    tokens = tokenize_and_pad(data)
    embeddings = encode_with_bert(tokens)
    return embeddings
text_data = train_df[['anchor_cleaned', 'target_cleaned']].values.tolist()
features = []
for text_pair in text_data:
    embeddings = extract_features(text_pair)
    features.append(embeddings)
features_tensor = torch.cat(features)
print(features_tensor)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\Hp\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hp\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyboardInterrupt: 

This code will tokenize the input text, pass it through the pre-trained BERT model, and extract the pooled output (CLS token) as the features for each input sentence.

Linear Regression, Ridge Regression, or Support Vector Regression (SVR). These models are suitable for predicting continuous values, which is what we have in the similarity scores.

In [ ]:
from sklearn.linear.model import LinearRegression
model=LinearRegression()

In [28]:
batch_size = 1000  # Define batch size

# Split the features and target (similarity scores)
X = features_tensor.numpy()  # Convert PyTorch tensor to NumPy array
y = train_df['score'].values

# Split the data into training and validation sets in smaller batches
X_train_batches = [X[i:i+batch_size] for i in range(0, len(X), batch_size)]
y_train_batches = [y[i:i+batch_size] for i in range(0, len(y), batch_size)]

# Process each batch separately
for X_batch, y_batch in zip(X_train_batches, y_train_batches):
    X_train, X_val, y_train, y_val = train_test_split(X_batch, y_batch, test_size=0.2, random_state=42)

    print("Training set shape:", X_train.shape)
    print("Validation set shape:", X_val.shape)


NameError: name 'features_tensor' is not defined